In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import re
import itertools
import time
import csv

# Function to extract category URLs
def get_cat(full_url):

    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15'
    path = '/Users/emma/Documents/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.get(full_url)

    # time.sleep(t)

    # Find class for tags with URLs to loop through
    elems = driver.find_elements_by_tag_name('a')
    url_list = [elem.get_attribute('href') for elem in elems]

    driver.quit()

    return url_list

In [2]:
# Get all URLs and remove NoneType values
men_list = get_cat('https://www.picture-organic-clothing.com/collection/en/68-men')
men_list = [string for string in men_list if string]

# Remove duplicate URLs
men_list = list(set(men_list))

# Get all URLs and remove NoneType values
women_list = get_cat('https://www.picture-organic-clothing.com/collection/en/3-women')
women_list = [string for string in women_list if string]

# Remove duplicate URLs
women_list = list(set(women_list))

# Merge all lists
picture_url_list = men_list + women_list

# Remove duplicate urls
picture_url_list = list(set(picture_url_list))

# Remove NoneType values and uneccssary items 
picture_url_list = [string for string in picture_url_list if string]
substring = ['book', 'wallet']
picture_url_list = [string for string in picture_url_list if all(sub not in string for sub in substring)]

print(len(picture_url_list))
#print(picture_url_list)

1333


In [3]:
url_list = []

# Scraper function
def picture_scraper(picture_url_list, url_list):

    # Define empty lists to store results and log failed URLs
    failed = []
    results = []

    for picture_url in tqdm(picture_url_list):
        if picture_url not in url_list:
            # Empty dictionary to store output
            picture_results = {}

            # Beautiful soup driver
            HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
            content = requests.get(picture_url, headers=HEADERS)
            soup = BeautifulSoup(content.text, 'html.parser')

            # Selenium driver
            user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'
            path = '/Users/emma/Documents/chromedriver'
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--window-size=1920,1080')
            options.add_argument('--ignore-certificate-errors')
            options.add_argument('--allow-running-insecure-content')
            options.add_argument(f'user-agent={user_agent}')
            driver = webdriver.Chrome(executable_path=path, options=options)
            driver.get(picture_url)

            try:
                # Product name
                picture_name = soup.find('h1', {'class': 'm-product__name'}).get_text(strip=True)
                picture_results['Name'] = picture_name

                # Product material
                picture_results['Material'] = soup.find('p', {'class': 'c-product-feature__desc'}).text.strip()

                # Product color
                picture_color = []
                
                for color in soup.find('ul', {'class': 'c-product-attribute-color'}).find_all('a'):
                    picture_color.append(color['title'])

                picture_results['Color'] = picture_color

                # Product price
                picture_price = soup.find('span', {'class': 'price'}).get_text(strip=True)
                picture_results['Price'] = picture_price
                
                # Product size
                picture_results['Size'] = [i.text.strip() for i in soup.find('select', {'class': 'm-product__attributes__item__select'}).find_all('option')]

                # Product URL
                picture_results['URL'] = picture_url

                # Product image
                picture_image = []
                color_image_dict = {}
                
                for img in soup.find_all('div', {'data-zoom': True}):
                    picture_image.append(img['data-zoom'])
                for color in picture_color:
                    for img in picture_image:
                        color_image_dict[color] = img

                picture_results['Image'] = color_image_dict

                # Product description
                picture_description = [desc.text for desc in driver.find_elements_by_xpath('//*[@id="product"]/div[1]/div/div/div[2]/div[1]/div/div[2]/div/div/div[2]')]
                picture_results['Description'] = picture_description

                # Add dictionary output to list
                results.append(picture_results)

            except (NoSuchElementException, AttributeError, TypeError):
                if NoSuchElementException:
                    failed.append(picture_url)
                elif AttributeError:
                    failed.append(picture_url)
                elif TypeError:
                    failed.append(picture_url)
                pass
    
            with open('picture_table.csv', mode='w') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=['Name', 'Material', 'Color', 'Price', 'Size', 'URL', 'Image', 'Description'])
                writer.writeheader()
                writer.writerows(results)

            with open('failed_picture.txt', mode='w') as txt_file:
                for row in failed:
                    txt_file.write(str(row) + '\n')

In [4]:
# Scraper
picture_scraper(picture_url_list, url_list)

100%|██████████| 1333/1333 [2:34:47<00:00,  6.97s/it] 


In [5]:
df = pd.read_csv('picture_table.csv')
df

,Name,Material,Color,Price,Size,URL,Image,Description
0,UNDER PT 20W,53% RECYCLED POLYESTER 47% POLYESTER,"['Red', 'Yellow', 'Army Green', 'Denim', 'Ketc...","139,99 €","['XS', 'S', 'M', 'L', 'XL', 'XXL']",https://www.picture-organic-clothing.com/colle...,{'Red': 'https://www.picture-organic-clothing....,['53% RECYCLED POLYESTER 47% POLYESTER\nRECYCL...
1,ANGLET TEE,60% Organic Cotton 40% Recycled Polyester,"['Pale Blue Melange', 'Crystal Pink Melange', ...","35,00 €","['S', 'M', 'L', 'XL', 'XXL']",https://www.picture-organic-clothing.com/colle...,{'Pale Blue Melange': 'https://www.picture-org...,['Jersey 160 GRSFront PrintFront Woven LabelBa...
2,BLOSSOM GRID FLEECE,50% POLYESTER 50% RECYCLED POLYESTER,"['Peonies Black', 'Turquoise', 'Terrazo Black']","62,99 €","['XS', 'S', 'M', 'L', 'XL']",https://www.picture-organic-clothing.com/colle...,{'Peonies Black': 'https://www.picture-organic...,['50% POLYESTER 50% RECYCLED POLYESTER\nRECYCL...
3,RONY ZIP TECH HOODIE,91% RECYCLED POLYESTER 9% ELASTANE,['Arctic blue'],"99,99 €","['S', 'M', 'L', 'XL', 'XXL']",https://www.picture-organic-clothing.com/colle...,{'Arctic blue': 'https://www.picture-organic-c...,['91% RECYCLED POLYESTER 9% ELASTANE\nRecycled...
4,SODAD CREW,B› 100% ORGANIC COTTON – A› 75% ORGANIC COTTON...,"['Grey Melange', 'Redwood']","79,99 €","['XS', 'S', 'M', 'L', 'XL']",https://www.picture-organic-clothing.com/colle...,{'Grey Melange': 'https://www.picture-organic-...,"['FLEECE 300 GRSCREW NECKRIBBED HEM, CUFFS & C..."
...,...,...,...,...,...,...,...,...
1136,ABSTRAL+ 2.5L PT,54% POLYESTER 46% RECYCLED POLYESTER,"['BLACK', 'Black ripstop']","119,99 €","['S', 'M', 'L', 'XL', 'XXL']",https://www.picture-organic-clothing.com/colle...,{'BLACK': 'https://www.picture-organic-clothin...,['54% POLYESTER 46% RECYCLED POLYESTER\nRecycl...
1137,LORELEI TECH TEE,93% RECYCLED POLYESTER 7% ELASTANE,"['Black', 'Ash rose']","45,00 €","['XS', 'S', 'M', 'L', 'XL']",https://www.picture-organic-clothing.com/colle...,{'Black': 'https://www.picture-organic-clothin...,['93% RECYCLED POLYESTER 7% ELASTANE\nRecycled...
1138,"CODE 19""",50% RECYCLED POLYESTER 38% POLYESTER 12% ELASTANE,"['Black', 'Brick', 'Palmtree']","45,50 €","['28', '30', '32', '34', '36', '38']",https://www.picture-organic-clothing.com/colle...,{'Black': 'https://www.picture-organic-clothin...,['FAST DRY 4D RECYCLED STRETCH FABRIC WELDED P...
1139,PURE Jacket,53% RECYCLED POLYESTER 47% POLYESTER,"['Yellow Dark Blue', 'Dark Blue Camel', 'Purpl...","195,99 €","['XS', 'S', 'M', 'L', 'XL', 'XXL']",https://www.picture-organic-clothing.com/colle...,{'Yellow Dark Blue': 'https://www.picture-orga...,['53% RECYCLED POLYESTER 47% POLYESTER\nRECYCL...
